# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [1]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI
import gradio as gr

In [2]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key looks good so far


In [3]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [4]:
ed = Website("https://edwarddonner.com")
ed.links

['https://edwarddonner.com/',
 'https://edwarddonner.com/connect-four/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/18/2025-ai-executive-briefing/',
 'https://edwarddonner.com/2025/05/18/2025-ai-executive-briefing/',
 'https://edwarddonner.com/2025/04/21/the-complete-agentic-ai-engineering-course/',
 'https://edwarddonner.com/2025/04/21/the-

## First step: Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [5]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [6]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}



In [7]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [8]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://edwarddonner.com - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
https://edwarddonner.com/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/
https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/
https://edwarddo

In [9]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [10]:
# Anthropic has made their site harder to scrape, so I'm using HuggingFace..

huggingface = Website("https://huggingface.co")
huggingface.links

['/',
 '/models',
 '/datasets',
 '/spaces',
 '/docs',
 '/enterprise',
 '/pricing',
 '/login',
 '/join',
 '/spaces',
 '/models',
 '/moonshotai/Kimi-K2-Instruct',
 '/HuggingFaceTB/SmolLM3-3B',
 '/THUDM/GLM-4.1V-9B-Thinking',
 '/black-forest-labs/FLUX.1-Kontext-dev',
 '/mistralai/Devstral-Small-2507',
 '/models',
 '/spaces/enzostvs/deepsite',
 '/spaces/black-forest-labs/FLUX.1-Kontext-Dev',
 '/spaces/kontext-community/kontext-relight',
 '/spaces/multimodalart/wan2-1-fast',
 '/spaces/FunAudioLLM/ThinkSound',
 '/spaces',
 '/datasets/fka/awesome-chatgpt-prompts',
 '/datasets/hackaprompt/Pliny_HackAPrompt_Dataset',
 '/datasets/HuggingFaceTB/smoltalk2',
 '/datasets/cais/hle',
 '/datasets/marcelbinz/Psych-101',
 '/datasets',
 '/join',
 '/pricing#endpoints',
 '/pricing#spaces',
 '/pricing',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/allenai',
 '/facebook',
 '/amazon',
 '/google',
 '/Intel',
 '/microsoft',
 '/grammarly',
 '/W

In [11]:
get_links("https://huggingface.co")

{'links': [{'type': 'about page', 'url': 'https://huggingface.co'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'},
  {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'},
  {'type': 'blog', 'url': 'https://huggingface.co/blog'},
  {'type': 'community forum', 'url': 'https://discuss.huggingface.co'},
  {'type': 'GitHub page', 'url': 'https://github.com/huggingface'},
  {'type': 'LinkedIn page',
   'url': 'https://www.linkedin.com/company/huggingface/'},
  {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [12]:
def get_all_details(url):
    result = "Landing page:\n" + url + "\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [13]:
print(get_all_details("https://huggingface.co"))

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/about'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'company page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'community page', 'url': 'https://huggingface.co/discuss'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}
Landing page:
https://huggingface.co
Webpage Title:
Hugging Face ‚Äì The AI community building the future.
Webpage Contents:
Hugging Face
Models
Datasets
Spaces
Community
Docs
Enterprise
Pricing
Log In
Sign Up
The AI community building the future.
The platform where the machine learning community collaborates on models, datasets, and applications.
Ex

In [14]:
# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."

system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Translate the brochure to Spangish, a mix of Spanish and English, with a humorous tone. \
Include details of company culture, customers and careers/jobs if you have the information."


In [15]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [16]:
print(get_brochure_user_prompt("HuggingFace", "https://huggingface.co"))

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'community page', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}
You are looking at a company called: HuggingFace
Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.
Landing page:
https://huggingface.co
Webpage Title:
Hugging Face ‚Äì The AI community building the future.
Webpage Contents:
Hugging Face
Models
Datasets
Spaces
C

In [17]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [18]:
create_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'community page', 'url': 'https://discuss.huggingface.co'}, {'type': 'github page', 'url': 'https://github.com/huggingface'}, {'type': 'linkedin page', 'url': 'https://www.linkedin.com/company/huggingface/'}, {'type': 'twitter page', 'url': 'https://twitter.com/huggingface'}]}


# Hugging Face - The AI Community That Hugs Back!

### Welcome to Hugging Face!

üåü‚ú® Where AI nerds unite and tech wizards conjure the magic of machine learning. Join the ONLY place where models can be hugged (virtually)!

---

## What We Offer

### Models
More than **1M models** ready to be discovered. That's right! We're like the candy shop of AI... except, you know, more productive. 

**Trending Models This Week:**
- **moonshotai/Kimi-K2-Instruct**: Don‚Äôt you dare bring this model to a moon landing ‚Äì it may just raise the bar too high!
- **HuggingFaceTB/SmolLM3-3B**: Small but mighty, like a chihuahua in a competition of Great Danes!
- **THUDM/GLM-4.1V-9B-Thinking**: Because thinking is hard and we might as well let AI do it for us.

### Spaces
We have more spaces than you‚Äôll find in your mom's attic!
- **DeepSite v2**: Create apps faster than you can say "machine learning".
- **FLUX.1 Kontext**: It's like image editing but with extra flair. 

### Datasets
With **250k+ datasets**, we‚Äôre basically running the Netflix of data! üé•üçø

---

## Company Culture

At Hugging Face, we believe in:
- **Collaboration**: Like a family reunion... but with fewer awkward questions about marital status.
- **Transparency**: We let you peek into the machinery of AI ‚Äì no smoke and mirrors! üîç
- **Innovation**: Constantly pushing the boundaries ‚Äì like trying to make a new recipe with the leftovers in the fridge.

üéâ **Career Opportunities**: We say no to stuffy offices! Join us and kick back, relax, and give AI a hug ‚Äì we mean it literally! Want to be part of something big? We've got jobs for all skills ‚Äì from code wizards to data sorcerers. 

---

## Our Customers

From Google to Microsoft, our clients practically form the Avengers of tech! ü¶∏‚Äç‚ôÇÔ∏è Imagine having their brains in a room... we guarantee it's less chaotic than it sounds!

---

## Join Us!

Come for the hugs, stay for the code! With products starting at **$0.60/hour for GPU** and team plans at **$20/user/month**, we‚Äôre always ready to welcome more curious minds into our AI family.

**Sign up today or we'll be forced to send you a rubber chicken!** üêî

---

## ¬°Bienvenido a Hugging Face en Spanglish!
**¬°La comunidad de IA que abraza!**

### ¬øQu√© ofrecemos?

üåü M√°s de **1M modelos** listos para ser descubiertos. ¬°Es como una tienda de caramelos de IA!

**Modelos de moda esta semana:**
- **moonshotai/Kimi-K2-Instruct**: ¬°No lo lleves a una luna! Podr√≠a dejar a todos boquiabiertos.
- **HuggingFaceTB/SmolLM3-3B**: Peque√±o pero poderoso, como un chihuahua en un concurso de grandes daneses.
- **THUDM/GLM-4.1V-9B-Thinking**: ¬°Porque pensar es dif√≠cil, dejemos que la IA lo haga!

### Espacios
¬°Tenemos m√°s espacios que los que hay en el √°tico de tu madre!
- **DeepSite v2**: Crea aplicaciones m√°s r√°pido que puedes decir "aprendizaje autom√°tico".
- **FLUX.1 Kontext**: Edici√≥n de im√°genes, pero con m√°s estilo.

### Conjuntos de Datos
Con **250k+ conjuntos de datos**, ¬°b√°sicamente somos el Netflix de los datos! üé•üçø

---

### Cultura de la Empresa

En Hugging Face:
- **Colaboraci√≥n**: Como una reuni√≥n familiar... ¬°pero con menos preguntas inc√≥modas sobre tu estado civil!
- **Transparencia**: Te dejamos ver la maquinaria de IA ‚Äì ¬°sin humo ni espejos!
- **Innovaci√≥n**: Empujando constantemente los l√≠mites, como intentar crear una nueva receta con sobras del refrigerador.

üéâ **Oportunidades Laborales**: ¬°Decimos no a las oficinas aburridas! √önete a nosotros y dale un abrazo a la IA ‚Äì lo decimos en serio. Desde magos del c√≥digo hasta hechiceros de datos, ¬°tenemos trabajos para todos!

---

### Nuestros Clientes

Desde Google hasta Microsoft, nuestros clientes forman los Vengadores de la tecnolog√≠a. ü¶∏‚Äç‚ôÇÔ∏è ¬°Imag√≠nate juntar sus cerebros en un cuarto! Garantizamos que es menos ca√≥tico de lo que parece.

---

### ¬°√önete a Nosotros!

Ven por los abrazos, qu√©date por el c√≥digo. Con productos starting from **$0.60/hora para GPU** y planes de equipo a **$20/usuario/mes**, siempre estamos listos para dar la bienvenida a m√°s mentes curiosas a nuestra familia de IA. 

**¬°Reg√≠strate hoy o nos veremos obligados a enviarte un pollo de goma!** üêî

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [19]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [20]:
#stream_brochure("HuggingFace", "https://huggingface.co")

In [21]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

#stream_brochure("HuggingFace", "https://huggingface.co")

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype. See what other students have done in the community-contributions folder -- so many valuable projects -- it's wild!</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 3 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!<br/>
            3. I'm trying out X/Twitter and I'm at <a href="https://x.com/edwarddonner">@edwarddonner<a> and hoping people will teach me how it's done..  
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>

In [22]:
def stream_model(prompt, model):
    if model=="GPT":
        result = stream_brochure("HuggingFace", "https://huggingface.co")
    elif model=="Claude":
        result = stream_brochure("HuggingFace", "https://huggingface.co")
    else:
        raise ValueError("Unknown model")
    yield from result

In [ ]:
view = gr.Interface(
    fn=stream_model,
    inputs=[gr.Textbox(label="Your message:"), gr.Dropdown(["GPT", "Claude"], label="Select model", value="GPT")],
    outputs=[gr.Markdown(label="Response:")],
    flagging_mode="never"
)
view.launch()

* Running on local URL:  http://127.0.0.1:7868
* To create a public link, set `share=True` in `launch()`.


Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'community page', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


# Hugging Face: Where AI Gets Its Cuddles

### Welcome to the Future! ü§ñüíï

At Hugging Face, we're not just throwing machine learning jargon around like confetti at a bot fiesta. Nope! We're the *AI community* bringing the future closer, one friendly model at a time‚Äîno awkward small talk required!

---

### Why Hugging Face? Because Models Don't Hug Themselves!

- **Explore 1M+ models!** Yeah, you read that right‚Äîover a million! It‚Äôs like the world's largest model convention, minus the bad coffee. ‚òï
- **Datasets galore!** Go ahead, browse through our collection of 250,000+ datasets. Think of it as a buffet for data enthusiasts‚Äîjust don't bring your own Tupperware! üçΩÔ∏è
- **AI Applications!** Need to generate an application? Say less; we have over 400,000 apps *(and none of them require you to refill a form under the fluorescent lights)*. 

---

### Join the Hugging Family‚ÄîSeriously, We Mean Family ü§ó

- **Culture:** Here at Hugging Face, we're not just co-workers; we're co-huggers! Emphasizing teamwork, passion, and a sprinkle of fun. Expect spontaneous lines of code and even more spontaneous karaoke nights! üé§
- **Community:** Meet your new pals! Our platform is home to developers, creatives, and anyone who loves AI‚Äîdon‚Äôt worry, we also tolerate humans! ü§ù
- **Careers:** We're always on the lookout for curious minds! Whether you‚Äôre a seasoned AI pioneer or just an enthusiast wearing nerd glasses, there‚Äôs a place for you in our Hugging corner of the Internet. üåê

---

### Meet Our Clients‚ÄîSo Many Friends, So Little Time! ‚è∞

- More than 50,000 organizations are cozying up with us, from cool startups to corporate giants like Amazon, Google, and Microsoft. They trust us, and guess what? You should too! 

---

### Discounts‚ÄîBecause Who Doesn't Love a Good Deal? üéâ

**Pricing starts from just $0.60/hour for GPU!** That‚Äôs cheaper than your last avocado toast! ü•ëüí∏

---

### Let's Hug It Out! üíñ

So, if you're ready to build the future with some awesome people (and tons of data), it's time to join us at Hugging Face!

---

### Spanglish Version - Hugging Face: Donde la IA se Abraza

# Hugging Face: Where AI se da Abrazos

### ¬°Bienvenidos al Futuro! ü§ñüíï

En Hugging Face, no solo lanzamos jerga de machine learning como confetti en una fiesta de robots. Nope! Somos la *comunidad AI* trayendo el futuro m√°s cerca, un modelo amigable a la vez‚Äî¬°sin charlas inc√≥modas!

---

### ¬øPor Qu√© Hugging Face? ¬°Porque los Modelos no se Abrazan Sola!

- **¬°Explora m√°s de 1M de modelos!** S√≠, lo le√≠ste bien‚Äî¬°m√°s de un mill√≥n! Es como la convenci√≥n de modelos m√°s grande del mundo, sin el mal caf√©. ‚òï
- **¬°Datasets por montones!** Adelante, navega a trav√©s de nuestra colecci√≥n de m√°s de 250,000 datasets. Piensa en ello como un buffet para los entusiastas de datos‚Äî¬°solo no traigas tu propio Tupperware! üçΩÔ∏è
- **¬°Aplicaciones AI!** ¬øNecesitas generar una aplicaci√≥n? Di menos; tenemos m√°s de 400,000 apps *(y ninguna de ellas requiere que rellenes un formulario bajo luces fluorescentes)*.

---

### √önete a la Familia Hugging‚ÄîEn serio, ¬°decimos Familia! ü§ó

- **Cultura:** Aqu√≠ en Hugging Face, no solo somos compa√±eros de trabajo; ¬°somos compis de abrazos! √ânfasis en trabajo en equipo, pasi√≥n y un toque de diversi√≥n. ¬°Esp√©rate l√≠neas de c√≥digo y noches de karaoke espont√°neas! üé§
- **Comunidad:** Conoce a tus nuevos amig@s. Nuestra plataforma es hogar de desarrolladores, creativos, y cualquiera que ama la IA‚Äî¬°no te preocupes, tambi√©n toleramos humanos! ü§ù
- **Carreras:** Siempre estamos buscando mentes curiosas. Ya seas un pionero en AI o solo un entusiasta con gafas nerd, hay un lugar para ti en nuestro rinc√≥n de abrazos en Internet. üåê

---

### Conoce a Nuestros Clientes‚Äî¬°Tantos Amigos, Tan Poco Tiempo! ‚è∞

M√°s de 50,000 organizaciones est√°n acurrucando con nosotros, desde cool startups hasta gigantes corporativos como Amazon, Google, y Microsoft. ¬°Ellos conf√≠an en nosotros, y adivina qu√©? ¬°T√∫ tambi√©n deber√≠as!

---

### Descuentos‚ÄîPorque ¬øA Qui√©n no le Gustan las Buenas Ofertas? üéâ

**Los precios empiezan desde solo $0.60/hora para GPU!** ¬°Eso es m√°s barato que tu √∫ltimo avocado toast! ü•ëüí∏

---

### ¬°Abracemos lo que Viene! üíñ

As√≠ que, si est√°s listo para construir el futuro con gente incre√≠ble (y toneladas de datos), ¬°es hora de unirte a nosotros en Hugging Face!

Traceback (most recent call last):
  File "/Users/jesusgarcia/.pyenv/versions/3.11.12/envs/llms/lib/python3.11/site-packages/gradio/queueing.py", line 625, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/jesusgarcia/.pyenv/versions/3.11.12/envs/llms/lib/python3.11/site-packages/gradio/route_utils.py", line 322, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/jesusgarcia/.pyenv/versions/3.11.12/envs/llms/lib/python3.11/site-packages/gradio/blocks.py", line 2147, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/jesusgarcia/.pyenv/versions/3.11.12/envs/llms/lib/python3.11/site-packages/gradio/blocks.py", line 1677, in call_function
    prediction = await utils.async_iteration(iterator)
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/jesusga